In [32]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from itertools import product
from skimage import measure
from math import sqrt
from sklearn.cluster import KMeans
import pandas as pd
import statistics
from scipy import ndimage

In [25]:
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

In [26]:
def dpm(membrane_img: str, fa_truth_binary: str, fa_pred_binary: str, fa_truth_contour: str, fa_pred_contour: str, weight_d: float, weight_s: float, weight_a: float):
    membrane = plt.imread(membrane_img)
    pixels = membrane.reshape(-1)
    xs = range(membrane.shape[0])
    ys = range(membrane.shape[1])
    indices = np.array(list(product(xs, ys)))
    pd.options.display.max_rows = 20
    index = pd.Series(pixels, name="pixels")
    df = pd.DataFrame({
        "xmem" : indices[:, 0],
        "ymem" : indices[:, 1]
    }, index=index)
    df = df[df.index != 0]
    df = df.reset_index()
    df = df.drop(columns=['pixels'])
    x_values = df['xmem'].values
    x_arr = np.reshape(x_values, (-1, 1))
    y_values = df['ymem'].values
    y_arr = np.reshape(y_values, (-1, 1))
    min_x = min(x_values)
    max_x = max(x_values)
    min_y = min(y_values)
    max_y = max(y_values)
    memb_arr = np.append(x_arr, y_arr, axis=1)
    coordinates = np.hstack((x_arr, y_arr))
    coord_add = np.zeros((10*(max_x - min_x), 2))
    count = 0
    coord_bound = []
    for x in range(min_x, max_x):
        new_coord = []
        for i in range(len(coordinates)):
            if coordinates[i][0] == x:
                new_coord.append(coordinates[i][1])
        y_min = min(new_coord)
        y_max = max(new_coord)
        coord_bound.append([x, y_min, y_max])


    fa_truth = cv2.imread(fa_truth_contour)
    ret,thresh = cv2.threshold(fa_truth,20,255,cv2.THRESH_BINARY_INV)
    labels= measure.label(thresh, background=0)
    bg_label = labels[0,0] 
    labels[labels==bg_label] = 0

    props = measure.regionprops(labels)

    centroids = np.zeros(shape=(len(np.unique(labels)) - 1,2))
    for i,prop in enumerate(props):
        my_centroid = prop.centroid
        centr = [0, 0]
        centr[0] = my_centroid[0]
        centr[1] = my_centroid[1]
        centroids[i,:]= centr

    kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42).fit(centroids)

    fa_pred = cv2.imread(fa_pred_contour)
    ret,thresh = cv2.threshold(fa_pred,20,255,cv2.THRESH_BINARY_INV)
    labels= measure.label(thresh, background=0)
    bg_label = labels[0,0] 
    labels[labels==bg_label] = 0

    props = measure.regionprops(labels)

    segmented_fa = cv2.imread(fa_pred_binary)
    segmented_fa_gray = rgb2gray(segmented_fa)
    gray = segmented_fa_gray*255
    gray = gray.astype(np.uint8)

    ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    pred_img = np.zeros((256,256))
    rect_coord = []
    rect_dim = []
    rect_angle = []
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        cv2.rectangle(pred_img, (x, y), (x+w, y+h), (255, 0, 0), -1)
        rect = cv2.minAreaRect(cntr)
        center, dim, angle = cv2.minAreaRect(cntr)
        rect_coord.append([x,y])
        rect_dim.append([w,h])
        rect_angle.append(angle)

    segmented_fa_truth = cv2.imread(fa_truth_binary)
    segmented_fa_truth_gray = rgb2gray(segmented_fa_truth)
    gray = segmented_fa_truth_gray*255
    gray = gray.astype(np.uint8)

    ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    new_img = np.zeros((256,256))
    rect_coord_truth = []
    rect_dim_truth = []
    rect_angle_truth = []
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        cv2.rectangle(new_img, (x, y), (x+w, y+h), (255, 0, 0), -1)
        rect = cv2.minAreaRect(cntr)
        center, dim, angle = cv2.minAreaRect(cntr)
        rect_coord_truth.append([x,y])
        rect_dim_truth.append([w,h])
        rect_angle_truth.append(angle)
    
    rect_img = np.zeros((256,256))
    angle_scores = []
    for i in range(len(rect_coord)):
        x_center = rect_coord[i][0] + rect_dim[i][0]//2
        y_center = rect_coord[i][1] + rect_dim[i][1]//2
        neighbors = get_neighbors(rect_coord_truth, rect_coord[i], 1)
        for neighbor in neighbors:
            for j in range(len(rect_coord_truth)):
                if neighbor[0] == rect_coord_truth[j][0] and neighbor[1] == rect_coord_truth[j][1]:
                    w = rect_dim_truth[j][0]
                    h = rect_dim_truth[j][1]
                    x = x_center - w//2
                    y = y_center - h//2
                    if rect_angle[i] > 90:
                        angle_pred = 180 - rect_angle[i]
                    elif -90 <= rect_angle[i] < 0:
                        angle_pred = abs(rect_angle[i])
                    elif rect_angle[i] < -90:
                        angle_pred = 180 - abs(rect_angle[i])
                    else:
                        angle_pred = rect_angle[i]
                    if rect_angle_truth[j] > 90:
                        angle_truth = 180 - rect_angle_truth[j]
                    elif -90 <= rect_angle_truth[j] < 0:
                        angle_truth = abs(rect_angle_truth[j])
                    elif rect_angle_truth[j] < -90:
                        angle_truth = 180 - abs(rect_angle_truth[j])
                    else:
                        angle_truth = rect_angle_truth[j]
                    angle_dev = abs(angle_pred - angle_truth)
                    angle_ind = 1 - (angle_dev/90)
                    angle_scores.append(angle_ind)
                    cv2.rectangle(rect_img, (x, y), (x+w, y+h), (255, 0, 0), -1)

    y_true = rect_img.flatten()
    y_true = y_true/255
    y_pred = pred_img.flatten()
    y_pred = y_pred/255
    tp = 0
    fn = 0
    fp = 0
    for k in range(len(y_true)):
        if y_true[k]==1 and y_pred[k]==1:
            tp += 1
        elif y_true[k]==1 and y_pred[k]==0:
            fn += 1
        elif y_true[k]==0 and y_pred[k]==1:
            fp += 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    s = (2*precision*recall)/(precision+recall)
    a = statistics.mean(angle_scores)
    centroids2 = np.zeros(shape=(len(np.unique(labels)) - 1,2))
    for i,prop in enumerate(props):
        my_centroid = prop.centroid
        centr = [0, 0]
        centr[0] = my_centroid[0]
        centr[1] = my_centroid[1]
        centroids2[i,:]= centr
    centroids_pred = []
    centroids_dropped = []
    for i in range(len(centroids2)):
        for j in range(len(coord_bound)):
            if int(centroids2[i][0]) == coord_bound[j][0] and coord_bound[j][1] <= int(centroids2[i][1]) <= coord_bound[j][2]:
                centroids_pred.append([centroids2[i][0], centroids2[i][1]])

    pred_clusters = kmeans.predict(centroids_pred)
    truth_clusters = kmeans.labels_

    unique_pred, counts_pred = np.unique(pred_clusters, return_counts=True)

    unique_truth, counts_truth = np.unique(truth_clusters, return_counts=True)

    clusters = []
    if len(counts_truth) != len(counts_pred):
        for j in range(len(counts_truth)-len(counts_pred)):
            counts_pred = np.append(counts_pred, 0)
    for i in range(len(counts_truth)):
        cluster_ind = (min(counts_truth[i], counts_pred[i]))/(max(counts_truth[i], counts_pred[i]))
        clusters.append(cluster_ind)

    dropped_number = len(centroids2) - len(centroids_pred)
    factor = 1 - (dropped_number/len(centroids2))

    d = factor*(statistics.mean(clusters))

    wd = weight_d
    ws = weight_s
    wa = weight_a
    dpm = (wd*d)+(ws*s)+(wa*a)
    
    return 'DPM: {}'.format(dpm), 'd: {}'.format(d), 's: {}'.format(s), 'a: {}'.format(a)

In [27]:
dpm('membrane_segmented.png', 'FA_truth_binary.png', 'FA_pred_binary.png', 'FA_truth_contours.png', 'FA_pred_contours.png', 0.33, 0.33, 0.33)

('DPM: 0.6619733175356031',
 'd: 0.6940542066629023',
 's: 0.6338492768332911',
 'a: 0.6780762666116946')

In [37]:
def dpm(ground_truth: str, prediction: str, size_threshold: int, scale: float, weight_d: float, weight_s: float, weight_a: float):
    
    def euclidean_distance(row1, row2):
        distance = 0.0
        for i in range(len(row1)-1):
            distance += (row1[i] - row2[i])**2
        return sqrt(distance)

    
    def get_neighbors(train, test_row, num_neighbors):
        distances = list()
        for train_row in train:
            dist = euclidean_distance(test_row, train_row)
            distances.append((train_row, dist))
        distances.sort(key=lambda tup: tup[1])
        neighbors = list()
        for i in range(num_neighbors):
            neighbors.append(distances[i][0])
        return neighbors
    
    
    def fa_processing(img_path: str, size_threshold: int, scale: float):
        img = cv2.imread(img_path)
        res = np.zeros((256, 256, 3))
        res[:,:,1] = img[:,:,1]
        filterSize =(3, 3)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize)
        tophat_img = cv2.morphologyEx(res[:,:,1], cv2.MORPH_TOPHAT, kernel)        
        processed_img = res
        processed_img[:,:,1] = tophat_img
        ret, bw_img = cv2.threshold(processed_img,10,255,cv2.THRESH_BINARY)
        gray = rgb2gray(bw_img)
        gray = gray * 255
        gray = gray.astype(np.uint8)
        ret, out_l = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
        contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        new_img = np.zeros((256,256))
        for cntr in contours:
            x,y,w,h = cv2.boundingRect(cntr)
            w = w*scale
            h = h*scale
            area = w*h
            if area >= size_threshold:
                cv2.drawContours(new_img, [cntr], 0, (255, 0, 0), -1)
        kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
        out_l = ndimage.convolve(new_img, kernel_laplace, mode='constant')
        out_l = out_l * 255
        ret, out_l = cv2.threshold(out_l,10,255,cv2.THRESH_BINARY)
        return new_img, out_l
    
    
    def membrane_processing(img_path: str):
        membrane = cv2.imread(img_path)
        ret, bw_img = cv2.threshold(membrane[:,:,2],10,255,cv2.THRESH_BINARY)
        res = np.zeros((256, 256, 3))
        res[:,:,2] = bw_img
        gray = rgb2gray(res)
        kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
        out_l = ndimage.convolve(gray, kernel_laplace, mode='constant')
        out_l = out_l * 255
        ret, out_l = cv2.threshold(out_l,10,255,cv2.THRESH_BINARY)
        return out_l
    
    
    membrane = membrane_processing(ground_truth)
    cv2.imwrite('membrane_segmented_test.png', membrane)
    pixels = membrane.reshape(-1)
    xs = range(membrane.shape[0])
    ys = range(membrane.shape[1])
    indices = np.array(list(product(xs, ys)))
    pd.options.display.max_rows = 20
    index = pd.Series(pixels, name="pixels")
    df = pd.DataFrame({
        "xmem" : indices[:, 0],
        "ymem" : indices[:, 1]
    }, index=index)
    df = df[df.index != 0]
    df = df.reset_index()
    df = df.drop(columns=['pixels'])
    x_values = df['xmem'].values
    x_arr = np.reshape(x_values, (-1, 1))
    y_values = df['ymem'].values
    y_arr = np.reshape(y_values, (-1, 1))
    min_x = min(x_values)
    max_x = max(x_values)
    min_y = min(y_values)
    max_y = max(y_values)
    memb_arr = np.append(x_arr, y_arr, axis=1)
    coordinates = np.hstack((x_arr, y_arr))
    coord_add = np.zeros((10*(max_x - min_x), 2))
    count = 0
    coord_bound = []
    for x in range(min_x, max_x):
        new_coord = []
        for i in range(len(coordinates)):
            if coordinates[i][0] == x:
                new_coord.append(coordinates[i][1])
        y_min = min(new_coord)
        y_max = max(new_coord)
        coord_bound.append([x, y_min, y_max])


    fa_truth = fa_processing(ground_truth, size_threshold, scale)[1]
    cv2.imwrite('FA_truth_contours_test.png', fa_truth)
    ret,thresh = cv2.threshold(fa_truth,20,255,cv2.THRESH_BINARY_INV)
    labels= measure.label(thresh, background=0)
    bg_label = labels[0,0] 
    labels[labels==bg_label] = 0

    props = measure.regionprops(labels)

    centroids = np.zeros(shape=(len(np.unique(labels)) - 1,2))
    for i,prop in enumerate(props):
        my_centroid = prop.centroid
        centr = [0, 0]
        centr[0] = my_centroid[0]
        centr[1] = my_centroid[1]
        centroids[i,:]= centr

    kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42).fit(centroids)

    fa_pred = fa_processing(prediction, size_threshold, scale)[1]
    ret,thresh = cv2.threshold(fa_pred,20,255,cv2.THRESH_BINARY_INV)
    labels= measure.label(thresh, background=0)
    bg_label = labels[0,0] 
    labels[labels==bg_label] = 0

    props = measure.regionprops(labels)

    segmented_fa = fa_processing(prediction, size_threshold, scale)[0]
    segmented_fa_gray = rgb2gray(segmented_fa)
    gray = segmented_fa_gray*255
    gray = gray.astype(np.uint8)

    ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    pred_img = np.zeros((256,256))
    rect_coord = []
    rect_dim = []
    rect_angle = []
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        cv2.rectangle(pred_img, (x, y), (x+w, y+h), (255, 0, 0), -1)
        rect = cv2.minAreaRect(cntr)
        center, dim, angle = cv2.minAreaRect(cntr)
        rect_coord.append([x,y])
        rect_dim.append([w,h])
        rect_angle.append(angle)

    segmented_fa_truth = fa_processing(ground_truth, size_threshold, scale)[0]
    segmented_fa_truth_gray = rgb2gray(segmented_fa_truth)
    gray = segmented_fa_truth_gray*255
    gray = gray.astype(np.uint8)

    ret, out_l = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(out_l, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    new_img = np.zeros((256,256))
    rect_coord_truth = []
    rect_dim_truth = []
    rect_angle_truth = []
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        cv2.rectangle(new_img, (x, y), (x+w, y+h), (255, 0, 0), -1)
        rect = cv2.minAreaRect(cntr)
        center, dim, angle = cv2.minAreaRect(cntr)
        rect_coord_truth.append([x,y])
        rect_dim_truth.append([w,h])
        rect_angle_truth.append(angle)
    
    rect_img = np.zeros((256,256))
    angle_scores = []
    for i in range(len(rect_coord)):
        x_center = rect_coord[i][0] + rect_dim[i][0]//2
        y_center = rect_coord[i][1] + rect_dim[i][1]//2
        neighbors = get_neighbors(rect_coord_truth, rect_coord[i], 1)
        for neighbor in neighbors:
            for j in range(len(rect_coord_truth)):
                if neighbor[0] == rect_coord_truth[j][0] and neighbor[1] == rect_coord_truth[j][1]:
                    w = rect_dim_truth[j][0]
                    h = rect_dim_truth[j][1]
                    x = x_center - w//2
                    y = y_center - h//2
                    if rect_angle[i] > 90:
                        angle_pred = 180 - rect_angle[i]
                    elif -90 <= rect_angle[i] < 0:
                        angle_pred = abs(rect_angle[i])
                    elif rect_angle[i] < -90:
                        angle_pred = 180 - abs(rect_angle[i])
                    else:
                        angle_pred = rect_angle[i]
                    if rect_angle_truth[j] > 90:
                        angle_truth = 180 - rect_angle_truth[j]
                    elif -90 <= rect_angle_truth[j] < 0:
                        angle_truth = abs(rect_angle_truth[j])
                    elif rect_angle_truth[j] < -90:
                        angle_truth = 180 - abs(rect_angle_truth[j])
                    else:
                        angle_truth = rect_angle_truth[j]
                    angle_dev = abs(angle_pred - angle_truth)
                    angle_ind = 1 - (angle_dev/90)
                    angle_scores.append(angle_ind)
                    cv2.rectangle(rect_img, (x, y), (x+w, y+h), (255, 0, 0), -1)

    y_true = rect_img.flatten()
    y_true = y_true/255
    y_pred = pred_img.flatten()
    y_pred = y_pred/255
    tp = 0
    fn = 0
    fp = 0
    for k in range(len(y_true)):
        if y_true[k]==1 and y_pred[k]==1:
            tp += 1
        elif y_true[k]==1 and y_pred[k]==0:
            fn += 1
        elif y_true[k]==0 and y_pred[k]==1:
            fp += 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    s = (2*precision*recall)/(precision+recall)
    a = statistics.mean(angle_scores)
    centroids2 = np.zeros(shape=(len(np.unique(labels)) - 1,2))
    for i,prop in enumerate(props):
        my_centroid = prop.centroid
        centr = [0, 0]
        centr[0] = my_centroid[0]
        centr[1] = my_centroid[1]
        centroids2[i,:]= centr
    centroids_pred = []
    centroids_dropped = []
    for i in range(len(centroids2)):
        for j in range(len(coord_bound)):
            if int(centroids2[i][0]) == coord_bound[j][0] and coord_bound[j][1] <= int(centroids2[i][1]) <= coord_bound[j][2]:
                centroids_pred.append([centroids2[i][0], centroids2[i][1]])

    pred_clusters = kmeans.predict(centroids_pred)
    truth_clusters = kmeans.labels_

    unique_pred, counts_pred = np.unique(pred_clusters, return_counts=True)

    unique_truth, counts_truth = np.unique(truth_clusters, return_counts=True)

    clusters = []
    if len(counts_truth) != len(counts_pred):
        for j in range(len(counts_truth)-len(counts_pred)):
            counts_pred = np.append(counts_pred, 0)
    for i in range(len(counts_truth)):
        cluster_ind = (min(counts_truth[i], counts_pred[i]))/(max(counts_truth[i], counts_pred[i]))
        clusters.append(cluster_ind)

    dropped_number = len(centroids2) - len(centroids_pred)
    factor = 1 - (dropped_number/len(centroids2))

    d = factor*(statistics.mean(clusters))

    wd = weight_d
    ws = weight_s
    wa = weight_a
    dpm = (wd*d)+(ws*s)+(wa*a)
    
    return 'DPM: {}'.format(dpm), 'd: {}'.format(d), 's: {}'.format(s), 'a: {}'.format(a)

In [39]:
dpm('unprocessed_ground_truth.png', 'prediction.png', 1, 0.7, 0.33, 0.33, 0.33)

<ipython-input-37-5fd751f5e0b2>:131: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  segmented_fa_gray = rgb2gray(segmented_fa)
<ipython-input-37-5fd751f5e0b2>:151: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  segmented_fa_truth_gray = rgb2gray(segmented_fa_truth)


('DPM: 0.6619733175356031',
 'd: 0.6940542066629023',
 's: 0.6338492768332911',
 'a: 0.6780762666116946')